<a href="https://colab.research.google.com/github/nariba/hello-pycuda/blob/main/2dim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install pycuda

     |████████████████████████████████| 1.6MB 4.5MB/s 
     |████████████████████████████████| 71kB 10.5MB/s 
     |████████████████████████████████| 81kB 10.0MB/s 
  Created wheel for pycuda: filename=pycuda-2020.1-cp36-cp36m-linux_x86_64.whl size=621567 sha256=7de128e1cfaba8aa30ca5569df9cc937421eb3e8e422e9b20d4ff4b081815dee
  Stored in directory: /root/.cache/pip/wheels/8f/78/d1/5bb826f81d9d490297a348d818ff3ee6dd6f2075b06dde6ea0
  Created wheel for pytools: filename=pytools-2020.4.3-py2.py3-none-any.whl size=61374 sha256=b4f9a17b46a91d3600e7fa1f22cdbcc40b661e6ec2d157a43686b1d8f452667e
  Stored in directory: /root/.cache/pip/wheels/af/c7/81/a22edb90b0b09a880468b2253bb1df8e9f503337ee15432c64
Successfully built pycuda pytools


In [2]:
import math
import numpy
import pycuda.gpuarray
from pycuda.compiler import SourceModule
import pycuda.autoinit

In [3]:
module = SourceModule("""
__global__ void add_two_array_2d(int nx, int ny, float *res, float *arr1, float *arr2){
  int x = threadIdx.x + blockDim.x * blockIdx.x;
  int y = threadIdx.y + blockDim.y * blockIdx.y;
  int ij = nx * y + x;
  if (x < nx && y < ny) {
    res[ij] = arr1[ij] + arr2[ij];
  }
}
""")

In [6]:
add_two_array = module.get_function("add_two_array_2d")

In [7]:
num_x, num_y = numpy.int32(5), numpy.int32(2)
num_components = num_x * num_y

In [8]:
arr1 = numpy.arange(num_components, dtype=numpy.float32).reshape(num_y, num_x)

In [9]:
numpy.random.seed(123)
arr2 = 10 * numpy.random.rand(num_y, num_x)
arr2 = arr2.astype(numpy.float32)

In [10]:
res = numpy.zeros([num_y, num_x], dtype=numpy.float32)

In [13]:
arr1_gpu = pycuda.gpuarray.to_gpu(arr1)
arr2_gpu = pycuda.gpuarray.to_gpu(arr2)
res_gpu = pycuda.gpuarray.to_gpu(res)

In [15]:
threads_per_block = (16, 16, 1)
block_x = math.ceil(num_x / threads_per_block[0])
block_y = math.ceil(num_y / threads_per_block[1])
blocks_per_grid = (block_x, block_y, 1)


In [17]:
add_two_array(num_x, num_y, res_gpu, arr1_gpu, arr2_gpu, block=threads_per_block, grid=blocks_per_grid)

In [19]:
res_gpu.get()

array([[ 6.9646916,  3.8613935,  4.2685146,  8.513147 , 11.19469  ],
       [ 9.231065 , 15.807642 , 13.848297 , 12.809319 , 12.921175 ]],
      dtype=float32)

In [20]:
arr1 + arr2

array([[ 6.9646916,  3.8613935,  4.2685146,  8.513147 , 11.19469  ],
       [ 9.231065 , 15.807642 , 13.848297 , 12.809319 , 12.921175 ]],
      dtype=float32)